In [1]:
import gc
import os
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress CUDA errors
import tensorflow as tf
from keras import mixed_precision, utils, layers, losses, optimizers, metrics, callbacks, Sequential
from sklearn import model_selection

mixed_precision.set_global_policy('mixed_float16') # https://keras.io/2/api/mixed_precision/policy/
utils.set_random_seed(42)

In [2]:
data = pl.read_csv('train.csv').drop('date_id').with_columns(
    pl.all().cast(pl.Float64, strict=False),
    pl.col('forward_returns').shift(-1).alias('lagged_forward_returns'),
    pl.col('risk_free_rate').shift(-1).alias('lagged_risk_free_rate'),
    pl.col('market_forward_excess_returns').shift(-1).alias('lagged_market_forward_excess_returns'),
).fill_null(0)
data

D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9,I1,I2,I3,I4,I5,I6,I7,I8,…,P4,P5,P6,P7,P8,P9,S1,S10,S11,S12,S2,S3,S4,S5,S6,S7,S8,S9,V1,V10,V11,V12,V13,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns,lagged_forward_returns,lagged_risk_free_rate,lagged_market_forward_excess_returns
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.002421,0.000301,-0.003038,-0.008495,0.000303,-0.009114
0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.008495,0.000303,-0.009114,-0.009624,0.000301,-0.010243
0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.009624,0.000301,-0.010243,0.004662,0.000299,0.004046
0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004662,0.000299,0.004046,-0.011686,0.000299,-0.012301
0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011686,0.000299,-0.012301,-0.006449,0.0003,-0.007066
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.565379,0.184524,0.01918,0.01918,0.005952,0.005952,0.911376,-0.083496,-0.572447,0.223638,-0.122314,1.20925,1.540116,1.655174,0.031415,0.33168,0.034722,0.038269,-0.301876,0.914683,0.27414,0.984115,0.080688,0.476521,0.597442,0.718254,0.238757,0.351852,…,0.51455,0.276769,-0.261326,0.811754,1.784929,0.039683,0.249933,0.273148,0.134921,0.634465,-0.446682,-0.052686,0.083995,0.055282,0.209656,0.409392,0.574661,0.748677,0.498677,-0.616395,0.561839,0.53373,-0.432282,0.785053,0.469577,0.837963,1.226772,0.822751,-0.707361,0.142857,-0.649616,0.002457,0.000155,0.00199,0.002312,0.000156,0.001845
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.562946,0.184193,0.018849,0.018849,0.005622,0.005622,0.911706,-0.083542,-0.57208,0.22291,-0.732397,1.225459,1.537761,1.672262,0.031085,0.331349,0.034392,0.038205,-0.301897,0.915013,0.269841,0.904453,0.073413,0.479167,0.605079,0.718254,0.220899,0.345238,…,0.379299,1.19926,-0.344274,0.690323,1.791596,0.037037,0.298533,0.933201,0.721561,1.211345,-0.11805,-0.249315,0.566799,0.10733,0.228175,0.409392,0.580932,0.37037,0.528439,-0.64204,0.587632,0.526455,-0.429506,0.767857,0.671958,0.837963,0.785877,0.805556,-0.715692,0.196098,-0.668289,0.002312,0.000156,0.001845,0.002891,0.000156,0.002424
0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.56052,0.183862,0.018519,0.018519,0.005291,0.005291,0.912037,-0.083874,-0.572016,0.222211,-0.800465,1.247273,1.534742,1.695469,0.030754,0.331019,0.034061,0.038118,-0.301918,0.915344,0.273148,0.842295,0.074074,0.478836,0.611319,0.724868,0.223

In [3]:
MAX_SIGNAL = 1.0
MIN_SIGNAL = 0.0
MID_SIGNAL = 0.5
OUTPUT_ACTIVATION = 'sigmoid'
MODEL_NAME = 'v44'
MAX_EPOCHS = 20000
MIN_EPOCHS = 5000

In [4]:
targets = np.where(data['forward_returns'] > data['risk_free_rate'], MAX_SIGNAL, MIN_SIGNAL)
targets

array([0., 0., 0., ..., 1., 1., 0.], shape=(8990,))

In [5]:
inputs = data.drop(['forward_returns', 'risk_free_rate', 'market_forward_excess_returns']).to_numpy()
inputs

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.49467704e-03,  3.02777778e-04, -9.11404562e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.62447844e-03,  3.01190476e-04, -1.02425375e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.66239748e-03,  2.99206349e-04,  4.04620350e-03],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         8.31020684e-03,  1.55952381e-04,  7.84287397e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         9.88397731e-05,  1.55873016e-04, -3.68167698e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
      shape=(8990, 97))

In [6]:
def hull_composite_loss(y_true, y_pred):
    penalties = [
        5.0 * losses.binary_crossentropy(y_true, y_pred, from_logits=True, label_smoothing=0.3),
        5.0 * tf.reduce_mean(tf.maximum(0.0, y_pred - y_true)), # (harshly) punishes false positives
        5.0 * tf.reduce_mean(tf.maximum(0.0, y_true - y_pred)), # (harshly) punishes false negatives
        1.0 * tf.reduce_mean(tf.square(tf.maximum(0.0, MIN_SIGNAL - y_pred))), # punishes values below min signal
        1.0 * tf.reduce_mean(tf.square(tf.maximum(0.0, y_pred - MAX_SIGNAL))), # punishes values above max signal
        # 1.0 * tf.reduce_mean(tf.square(1.0 / (tf.abs(y_pred - MID_SIGNAL) + 1e7))), # punishes indecision (predictions halfway between min and max signal)
        # 1.0 / (tf.math.reduce_variance(y_pred) + 1e7), # punishes guessing constant or near-constant values
        # 1.0 / (tf.math.reduce_std(y_pred) + 1e7), # also punishes guessing constant or near-constant values
    ]
    return sum(penalties)

In [7]:
N_FEATURES = inputs.shape[-1]
N_FEATURES

97

In [8]:
model = Sequential([
    layers.Input([N_FEATURES]),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(N_FEATURES),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(N_FEATURES),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(int(2 * N_FEATURES // 3)),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(int(2 * N_FEATURES // 3)),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(int(4 * N_FEATURES // 9)),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(int(4 * N_FEATURES // 9)),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(int(16 * N_FEATURES // 81)),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(int(16 * N_FEATURES // 81)),
    layers.Dropout(0.1),
    layers.BatchNormalization(),
    layers.Dense(1, activation=OUTPUT_ACTIVATION)
], name=MODEL_NAME)

model.compile(loss=hull_composite_loss, optimizer=optimizers.Adamax(), metrics=[metrics.MeanAbsoluteError()])

I0000 00:00:1759958124.590502  631662 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9410 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1759958124.590894  631662 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 4723 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5


In [9]:
for k, (i_train, i_val) in enumerate(model_selection.TimeSeriesSplit().split(inputs)):
    model.fit(inputs[i_train], targets[i_train], epochs=MAX_EPOCHS, validation_data=(inputs[i_val], targets[i_val]), verbose=0,
              callbacks=[callbacks.EarlyStopping(patience=MIN_EPOCHS, restore_best_weights=True), callbacks.TerminateOnNaN()])
    model.save(f'.models/{model.name}.keras')
    while gc.collect() > 0: pass

I0000 00:00:1759958131.840635  631830 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


KeyboardInterrupt: 

In [ ]:
test_data = pl.read_csv('test.csv').drop(['date_id', 'is_scored']).fill_null(0)
test_data

In [ ]:
test_targets = np.where(test_data['lagged_market_forward_excess_returns'].shift(1) > 0, MAX_SIGNAL, MIN_SIGNAL)
test_targets

In [ ]:
test_inputs = test_data.to_numpy()
test_inputs.shape

In [ ]:
plt.figure(figsize=(13, 5))
plt.title(f'{model.name} Signals vs Hull Targets')
plt.plot(test_targets, zorder=-1)
indices = np.arange(test_targets.shape[0])
plt.scatter(indices, test_targets, label='Target', edgecolors='k', c='g', s=64)
signals = model.predict(test_inputs)
plt.plot(signals, zorder=-1)
plt.scatter(indices, signals, label='Signal', marker='X', edgecolors='k', c='r', s=64)
plt.legend()
pass

In [ ]:
signals